# Content Based Filtering
- 영화의 줄거리를 이용해, TF-IDF를 계산한뒤, TF-IDF 벡터간의 유사도를 구해 비슷한 영화를 추천
- https://wikidocs.net/24603

## 1. 데이터 Load

In [ ]:
import csv
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [ ]:

movies = pd.read_csv("movie_doc.csv",sep="\t")
movies.tail()

In [ ]:
stop_words = []
with open('korean_stopwords.txt', encoding="utf-8") as f:
    for line in f.readlines():
        row = line.strip().split()[0]
        stop_words.append(row)

## 2. Tokenizer 정의

In [ ]:
from konlpy.tag import Okt
okt = Okt()


In [ ]:
pd.options.mode.chained_assignment = None
np.random.seed(0)

from konlpy.tag import Okt
okt = Okt()

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def tokenizer(raw, pos=["Noun"], stopword=stop_words+[]):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

# 테스트 문장
rawdata = movies['story'].tolist()

## 3. TF-IDF 행렬 계산

In [ ]:

vectorize = TfidfVectorizer(
    tokenizer=tokenizer,
    min_df=0.01, #TODO : 특정 단어가 최소 등장해야하는 문서의 수 = 이 이하 등장 하는 단어는 무시
    max_df=0.99, #TODO : 특정 단어가 최대 등장해야하는 문서의 수 = 이 이상 등장 하는 단어는 무시
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)
X = vectorize.fit_transform(rawdata)

print(
    'fit_transform, (sentence {}, feature {})'.format(X.shape[0], X.shape[1])
)


print(X.toarray())



# 문장에서 뽑아낸 feature 들의 배열
features = vectorize.get_feature_names()


In [ ]:
tf_idf_mtx = pd.DataFrame(X.toarray(), columns = features)

In [ ]:
tf_idf_mtx.head()

## 4. Cosine 유사도 계산

In [ ]:
cosine_sim = linear_kernel(tf_idf_mtx, tf_idf_mtx)

In [ ]:
cosine_sim

In [ ]:
indices = pd.Series(movies.index, index=movies['title'])
print(indices.head())

## 5. 비슷한 영화 추천

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return movies['title'].iloc[movie_indices]


In [ ]:
get_recommendations('닥터 스트레인지')


In [ ]:
type(indices)

## 6. 관련 데이터 저장

In [ ]:
with open('sim.npy', 'wb') as f:
    np.save(f, cosine_sim)

In [ ]:
with open('sim.npy', 'rb') as f:
    cosine_sim = np.load(f)
    print(cosine_sim)

In [ ]:
indices.to_csv("index.csv")

In [ ]:
indices = pd.read_csv("index.csv", header = None, index_col = 0, squeeze = True)

In [ ]:
indices["암살"]